# Storm Typing Results Notebook

In [7]:
from IPython.display import HTML
from notebook_utils import (
    get_notebook_paths,
    process_notebooks,
    convert_paths_to_html_links,
)

notebooks_folder_path = "notebooks"
notebook_paths = get_notebook_paths(notebooks_folder_path)
all_data = process_notebooks(notebook_paths)
all_data_with_links = convert_paths_to_html_links(all_data, notebook_paths)

## Mid-Latitude Cyclone

In [8]:
HTML(
    all_data[all_data["likely_storm_type"] == "mid_latitude_cyclone"].to_html(
        escape=False
    )
)

,storm_start_date,local_storm,mesoscale_storm,mid_latitude_cyclone,tropical,likely_storm_type,notes,HTML_Link
0,1986_03_12,0.0,1.0,6.0,0.0,mid_latitude_cyclone,,notebooks/storm-1986_03_12.html
2,2015_10_23,0.0,2.0,6.0,1.0,mid_latitude_cyclone,Non-tropical low develops just off coast,notebooks/storm-2015_10_23.html


## Mesoscale Convective

In [9]:
HTML(all_data[all_data["likely_storm_type"] == "mesoscale_storm"].to_html(escape=False))

,storm_start_date,local_storm,mesoscale_storm,mid_latitude_cyclone,tropical,likely_storm_type,notes,HTML_Link
1,1989_05_13,3.0,6.0,1.0,0.0,mesoscale_storm,Too much precip coverage to be local storm,notebooks/storm-1989_05_13.html


## Tropical

In [10]:
HTML(all_data[all_data["likely_storm_type"] == "tropical"].to_html(escape=False))

,storm_start_date,local_storm,mesoscale_storm,mid_latitude_cyclone,tropical,likely_storm_type,notes,HTML_Link
3,2017_08_27,0.0,0.0,1.0,6.0,tropical,Tropical Cyclone Harvey,notebooks/storm-2017_08_27.html


## Local Storms

In [11]:
HTML(all_data[all_data["likely_storm_type"] == "local_storm"].to_html(escape=False))

,storm_start_date,local_storm,mesoscale_storm,mid_latitude_cyclone,tropical,likely_storm_type,notes,HTML_Link


## Hybrid

In [12]:
HTML(all_data[all_data["likely_storm_type"] == "hybrid"].to_html(escape=False))

,storm_start_date,local_storm,mesoscale_storm,mid_latitude_cyclone,tropical,likely_storm_type,notes,HTML_Link
